![banner](https://anopheles-genomic-surveillance.github.io/_images/banner.jpg)

***Training course in data analysis for genomic surveillance of African malaria vectors - Workshop 5***

---

# Module 3 - Genetic diversity summary statistics

**Theme: Analysis**

This module covers how to compute genetic diversity summary statistics, which are a convenient and informative way to quantify and compare diversity in different cohorts. 

## Learning objectives

At the end of this module you will be able to:

* Select cohorts for comparative analysis of genetic diversity
* Compute nucleotide diversity, Watterson's estimator and Tajima's D
* Plot diversity statistics for multiple cohorts
* Interpret the results

## Lecture

### English

In [1]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/ohKfiu_WC-g" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

### Français

In [2]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/pkwpz8cWp44" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

## Setup

Install and import the packages we'll need.

In [1]:
!pip install -q malariagen_data

In [2]:
import malariagen_data
import plotly.express as px

Here we will compute summary statistics using data from millions of SNPs. This can take some time, so we will use Google Drive to save results and avoid having to rerun computations. To do this we first need to mount Google Drive to our colab VM:

In [3]:
try:
    from google.colab import drive
    drive.mount("drive")
except ImportError:
    pass

Configure access to MalariaGEN data in Google Cloud.

In [4]:
ag3 = malariagen_data.Ag3(
    results_cache="drive/MyDrive/malariagen_data_cache"
)
ag3

<MalariaGEN Ag3 API client>
Storage URL             : gs://vo_agam_release/
Data releases available : 3.0
Results cache           : /home/jovyan/github/anopheles-genomic-surveillance/anopheles-genomic-surveillance.github.io/docs/workshop-5/drive/MyDrive/malariagen_data_cache
Cohorts analysis        : 20220608
Species analysis        : aim_20220528
Site filters analysis   : dt_20200416
Software version        : malariagen_data 6.0.0
Client location         : Iowa, US (Google Cloud)
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact data@malariagen.net. For API documentation see 
https://malariagen.github.io/vector-data/ag3/api.html

## Selecting cohorts

In this module we will be comparing genetic diversity between mosquitoes representing different geographical locations, time points and species. Before we begin any comparative analysis of genetic diversity, we first need to decide how to group the mosquitoes we've sequenced into cohorts. Each cohort we define will represent a sample of mosquitoes from a given time, place and species.

Let's first browse the different sample locations for which we have data. In [workshop 1, module 2](../workshop-1/module-2-sample-metadata) we looked at how to create an interactive map with sampling locations using ipyleaflet. Because this is something we want to do often, we've added a convenience function `plot_samples_interactive_map()` to do this for you. This will plot a map showing sampling locations for all available samples.

In [5]:
ag3.plot_samples_interactive_map()

Load sample metadata:   0%|          | 0/28 [00:00<?, ?it/s]

Map(center=[-2, 20], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

For the purposes of this module we will begin by focusing on Tanzania. To get more information on the numbers of samples available from Tanzania, we can use pandas to create a pivot table as we also illustrated in [workshop 1, module 2](workshop-1/module-2-sample-metadata). Again, because we do this often, we've created a convenience function `count_samples()`.

In [6]:
ag3.count_samples(
    sample_query="country == 'Tanzania'"
)

taxon                                             arabiensis  gambiae  gcx3
country  admin1_iso admin1_name admin2_name year                           
Tanzania TZ-05      Kagera      Muleba      2015         137       32     1
         TZ-13      Mara        Tarime      2012          47        0     0
         TZ-25      Tanga       Muheza      2013           1       32    10
         TZ-26      Manyara     Moshi       2012          40        0     0

When grouping samples by geographical location, we sometimes can choose whether to aggregate data from nearby locations, or whether to keep them separate. In general, we want the finest spatial granularity possible, but we also need enough samples to make reasonable estimates of genetic diversity, and so we might have to make a trade-off between granularity and sample size. 

Similarly, when grouping samples by date of collection, we can choose whether to aggregate data from the same month, season or year. Again, in general we would prefer the finest temporal granularity possible, but might also need sufficient sample size.

For computing diversity summary statistics, there is no hard-and-fast rule about how many samples are needed, but a sample size of 10 is probably the bare minimum we would consider, and we would prefer more if possible. 

To simplify grouping of mosquitoes into cohorts, we have predefined some metadata columns which provide cohort labels at different levels of granularity. Let's take a peek at the sample metadata:

In [7]:
df_samples_tz = ag3.sample_metadata(
    sample_query="country == 'Tanzania'"
)
df_samples_tz.head()

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,...,aim_species,country_iso,admin1_name,admin1_iso,admin2_name,taxon,cohort_admin1_year,cohort_admin1_month,cohort_admin2_year,cohort_admin2_month
0,BL0046-C,Plate_C_H6,Bilali Kabula,Tanzania,Muleba,2015,6,-1.962,31.621,F,...,intermediate_gambiae_coluzzii,TZA,Kagera,TZ-05,Muleba,gcx3,TZ-05_gcx3_2015,TZ-05_gcx3_2015_06,TZ-05_Muleba_gcx3_2015,TZ-05_Muleba_gcx3_2015_06
1,BL0047-C,Plate_F_D4,Bilali Kabula,Tanzania,Muleba,2015,3,-1.962,31.621,F,...,arabiensis,TZA,Kagera,TZ-05,Muleba,arabiensis,TZ-05_arab_2015,TZ-05_arab_2015_03,TZ-05_Muleba_arab_2015,TZ-05_Muleba_arab_2015_03
2,BL0048-C,Plate_F_E4,Bilali Kabula,Tanzania,Muleba,2015,3,-1.962,31.621,F,...,arabiensis,TZA,Kagera,TZ-05,Muleba,arabiensis,TZ-05_arab_2015,TZ-05_arab_2015_03,TZ-05_Muleba_arab_2015,TZ-05_Muleba_arab_2015_03
3,BL0049-C,Plate_F_F4,Bilali Kabula,Tanzania,Muleba,2015,3,-1.962,31.621,F,...,arabiensis,TZA,Kagera,TZ-05,Muleba,arabiensis,TZ-05_arab_2015,TZ-05_arab_2015_03,TZ-05_Muleba_arab_2015,TZ-05_Muleba_arab_2015_03
4,BL0050-C,Plate_F_G4,Bilali Kabula,Tanzania,Muleba,2015,3,-1.962,31.621,F,...,arabiensis,TZA,Kagera,TZ-05,Muleba,arabiensis,TZ-05_arab_2015,TZ-05_arab_2015_03,TZ-05_Muleba_arab_2015,TZ-05_Muleba_arab_2015_03


If you scroll to the right, you'll see columns named "cohort_admin1_year", "cohort_admin1_month", "cohort_admin2_year" and "cohort_admin2_month". Each of these columns provides a different set of cohort labels. All are grouped by species (taxon).

For this analysis, let's use the "cohort_admin2_year" column to group samples by administrative level 2 divisions (usually called districts) and year. Here are the cohort labels for the Tanzania samples and the sample sizes.

In [8]:
df_samples_tz.groupby("cohort_admin2_year").size()

cohort_admin2_year
TZ-05_Muleba_arab_2015    137
TZ-05_Muleba_gamb_2015     32
TZ-05_Muleba_gcx3_2015      1
TZ-13_Tarime_arab_2012     47
TZ-25_Muheza_arab_2013      1
TZ-25_Muheza_gamb_2013     32
TZ-25_Muheza_gcx3_2013     10
TZ-26_Moshi_arab_2012      40
dtype: int64

We can see there are two cohorts with only 1 sample - these will get dropped from our analysis. Otherwise, all cohorts have at least 10 samples.

## Recap: genetic diversity summary statistics

We are going to compute the following three summary statistics for each cohort:

* Nucleotide diversity (`theta_pi`) - This quantifies the average number of differences (mismatches) between any pair of DNA sequences in a cohort. The higher this statistic, the more differences you will expect to see between any pair of randomly sampled DNA sequences.
* Watterson's estimator (`theta_w`) - This quantifies the number of segregating sites within a cohort, scaled by a constant. The higher this statistic, the more sites are found to be segregating (i.e., polymorphic).
* Tajima's D (`tajima_d`) - This quantifies the difference between Watterson's estimator and nucleotide diversity, and scales the result by a constant. This statistic is useful because it focuses not on the magnitude but on the architecture of diversity in a cohort, compared to a neutrally evolving population of constant size. Negative values imply an excess of rare alleles, and positive values imply a deficit of rare alleles, relative to a population at equilibrium. 

## Computing diversity summary statistics

Let's begin by computing these summary statistics in a single cohort. For this we can use the function `cohort_diversity_stats()`. Let's look at the docstring.

In [9]:
ag3.cohort_diversity_stats?

Let's now compute the statistics for one of our Tanzanian cohorts, TZ-05_Muleba_gamb_2015. Don't worry about the other parameters for now, we'll investigate them shortly.

In [10]:
stats = ag3.cohort_diversity_stats(
    cohort="TZ-05_Muleba_gamb_2015",
    cohort_size=30,
    region="3L:15,000,000-41,000,000",
    site_mask="gamb_colu_arab",
    site_class="CDS_DEG_4",
)
stats

KeyboardInterrupt: 

In [ ]:
stats = ag3.cohort_diversity_stats(
    cohort="TZ-05_Muleba_gamb_2015",
    cohort_size=10,
    region="3L:15,000,000-41,000,000",
    site_mask="gamb_colu_arab",
    site_class="CDS_DEG_4",
)
stats

In [ ]:
stats = ag3.cohort_diversity_stats(
    cohort="TZ-05_Muleba_gamb_2015",
    cohort_size=10,
    region="3L:15,000,000-41,000,000",
    site_mask="gamb_colu_arab",
    site_class="CDS_DEG_0",
)
stats

## Comparing diversity between cohorts

### Example: Tanzania

In [ ]:
df_samples_tz.groupby("cohort_admin2_year").size()

In [ ]:
ag3.diversity_stats?

In [ ]:
df_stats_tz_admin2_year = ag3.diversity_stats(
    sample_query="country == 'Tanzania'",
    cohorts="admin2_year",
    cohort_size=10,
    region="3L:15,000,000-41,000,000",
    site_mask="gamb_colu_arab",
    site_class="CDS_DEG_4",
)
df_stats_tz_admin2_year

In [ ]:
df_stats_tz_admin2_year.columns

In [ ]:
px.bar(
    data_frame=df_stats_tz_admin2_year,
    x="cohort",
    y="theta_pi_estimate",
    error_y="theta_pi_ci_err",
    color="taxon",
    height=450,
    width=500,
)

In [ ]:
def plot_diversity_stats(
    df_stats, 
    color=None, 
    height=450, 
    template="plotly_white"
):

    # set up common plotting parameters
    hover_name = "cohort"
    hover_data = [
        "taxon",
        "country",
        "admin1_iso",
        "admin1_name",
        "admin2_name",
        "year",
        "month",
    ]
    labels = {
        'theta_pi_estimate': r'$\widehat{\theta}_{\pi}$',
        'theta_w_estimate': r'$\widehat{\theta}_{w}$',
        'tajima_d_estimate': r'$D$',
        'cohort': "Cohort",
        'taxon': 'Taxon',
        'country': "Country",
    }
    width = 300 + 30 * len(df_stats)

    # nucleotide diversity bar plot
    fig = px.bar(
        data_frame=df_stats,
        x="cohort",
        y="theta_pi_estimate",
        error_y="theta_pi_ci_err",
        title="Nucleotide diversity",
        color=color,
        height=height,
        width=width,
        hover_name=hover_name,
        hover_data=hover_data,
        labels=labels,
        template=template,
    )
    fig.show()

    # watterson estimator bar plot
    fig = px.bar(
        data_frame=df_stats,
        x="cohort",
        y="theta_w_estimate",
        error_y="theta_w_ci_err",
        title="Watterson estimator",
        color=color,
        height=height,
        width=width,
        hover_name=hover_name,
        hover_data=hover_data,
        labels=labels,
        template=template,
    )
    fig.show()

    # tajima's d bar plot
    fig = px.bar(
        data_frame=df_stats,
        x="cohort",
        y="tajima_d_estimate",
        error_y="tajima_d_ci_err",
        title="Tajima's D",
        color=color,
        height=height,
        width=width,
        hover_name=hover_name,
        hover_data=hover_data,
        labels=labels,
        template=template,
    )
    fig.show()

    # scatter plot comparing diversity estimators
    fig = px.scatter(
        data_frame=df_stats,
        x="theta_pi_estimate",
        y="theta_w_estimate",
        error_x="theta_pi_ci_err",
        error_y="theta_w_ci_err",
        title="Diversity estimators",
        color=color,
        width=500,
        height=500,
        hover_name=hover_name,
        hover_data=hover_data,
        labels=labels,
        template=template,
    )
    fig.show()

In [ ]:
plot_diversity_stats(df_stats_tz_admin2_year, color="taxon")

**Exercise 1**

Analyse Tanzania by admin2_month...

In [ ]:
# df_samples_tz.groupby("cohort_admin2_month").size()

In [ ]:
# df_stats_tz_admin2_month = ag3.diversity_stats(
#     cohorts="admin2_month",
#     cohort_size=10,
#     region="3L:15,000,000-41,000,000",
#     site_mask="gamb_colu_arab",
#     site_class="CDS_DEG_4",
#     sample_query="country == 'Tanzania'",
# )
# plot_diversity_stats(df_stats_tz_admin2_month, color="taxon")

### Example: East Africa


In [ ]:
ag3.plot_samples_interactive_map()

In [ ]:
ag3.count_samples(
    sample_query="country in ['Tanzania', 'Kenya', 'Uganda']"
)

In [ ]:
ag3.sample_metadata(
    sample_query="country in ['Tanzania', 'Kenya', 'Uganda']"
).groupby("cohort_admin2_year").size()

In [ ]:
df_stats_tz_ke_ug_admin2_year = ag3.diversity_stats(
    sample_query="country in ['Tanzania', 'Kenya', 'Uganda']",
    cohorts="admin2_year",
    cohort_size=10,
    region="3L:15,000,000-41,000,000",
    site_mask="gamb_colu_arab",
    site_class="CDS_DEG_4",
)
df_stats_tz_ke_ug_admin2_year

In [ ]:
plot_diversity_stats(df_stats_tz_ke_ug_admin2_year, color="taxon")

**Exercise 2**

Add Mozambique and Malawi to the East Africa analysis...

In [ ]:
# df_stats_east_admin2_year = ag3.diversity_stats(
#     cohorts="admin2_year",
#     cohort_size=10,
#     region="3L:15,000,000-41,000,000",
#     site_mask="gamb_colu_arab",
#     site_class="CDS_DEG_4",
#     sample_query="country in ['Tanzania', 'Kenya', 'Uganda', 'Mozambique', 'Malawi']",
# )
# plot_diversity_stats(df_stats_east_admin2_year, color="taxon")

### Example: Burkina Faso

In [ ]:
ag3.count_samples(
    sample_query="country == 'Burkina Faso'"
)

**Exercise 3**

Analysis BF. Compare gambiae and coluzzii. Compare 2012 and 2014.

In [ ]:
# df_stats_bf_admin2_year = ag3.diversity_stats(
#     cohorts="admin2_year",
#     cohort_size=13,
#     region="3L:15,000,000-41,000,000",
#     site_mask="gamb_colu_arab",
#     site_class="CDS_DEG_4",
#     sample_query="country == 'Burkina Faso'",
# )
# plot_diversity_stats(df_stats_bf_admin2_year, color="taxon")

## Well done!

Rerun notebook, complete exercises you find.

When finished, choose any country and perform diversity analysis. Share your findings and conclusions with colleagues.